# Import and init GEE

In [1]:
import ee, eemont
import geemap
import geemap.colormaps as geecm
import operator

In [2]:
#ee.Authenticate()
ee.Initialize()

In [3]:
from pyce import gee
from pyce.tools.lc_mapping import LandCoverMap

# 0. Extract alpine LC over French area

In [70]:
fc_FR = ee.FeatureCollection('users/aguerou/ice_and_life/erable_2025/data_ancillary/alpes_francaises')

## 0.1 Create asset over BVs

In [71]:
lc_alpine = ee.Image("users/aguerou/ice_and_life/leca_projects/carto_h1b_alpine_conv")
rgi = ee.FeatureCollection("users/aguerou/ice_and_life/carto_h1b/lia_shp/c3s_gi_rgi11_s2_2015_v2")

In [72]:
scale = 20
projection = "EPSG:3035"

## 0.2 Merge RGI and Landcover data

In [73]:
# Get mask of glacier
glacier = lc_alpine.clip(fc_FR).clip(rgi)

In [74]:
# Put value to glacier pixel
classif_FR = lc_alpine.clip(fc_FR).where(glacier.gt(-1), 4)

### Plot to check

In [75]:
Map = geemap.Map()
Map.centerObject(fc_FR, 12)
Map.addLayer(classif_FR, {"min": -1, "max": 6, "palette": [
    '#ffffff',
    '#a5763b',
    'lightgreen',
    'purple',
    'darkgreen',
    'lightblue',
    'blue',
    'yellow']}, "CLASSIF")
Map

Map(center=[44.74432688341291, 6.189615253288755], controls=(WidgetControl(options=['position', 'transparent_b…

## 0.3 Export as one asset

In [79]:
geemap.ee_export_image_to_drive(classif_FR,
                                folder="gee_dwnld_lacs_sentinelles",
                                fileNamePrefix="carto_landcover_s2_alps_fr",
                                scale=scale,
                                crs=projection,
                                maxPixels=5e9,
                                region=ee.Geometry.convexHull(fc_FR.geometry()),
                                formatOptions={"noData": -1})

# 1. Create indices over region

## 1.1 Parameters

Indices parameters

In [42]:
# L'année indiquée n'a pas d'importance / on veut slmt le day_of_year
day_beg = ee.Date('2017-07-15')
day_end = ee.Date('2017-10-15')

In [43]:
indices_classif = ["NARI", "NCRI"]

Scale and projection

In [44]:
scale = 20
projection = "EPSG:3035"
cloud_thresh = 60

ROI shapefile

In [45]:
fc_corse = ee.FeatureCollection('users/aguerou/lacs_sentinelles/ancillary/regions-20180101').filter(ee.Filter.eq('nom', 'Corse'))
fc_pyr = ee.FeatureCollection('users/aguerou/lacs_sentinelles/ancillary/pyrenees')

In [50]:
# Set projection and scale to avoid having multiple ones since GLO30 is an ImageCollection - apparently needed for slope computation
# See https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_DEM_GLO30
dem_pyr = ee.ImageCollection("COPERNICUS/DEM/GLO30").filterBounds(fc_pyr).select("DEM")
dem_pyr = dem_pyr.mosaic().setDefaultProjection(crs='EPSG:3035',scale=30).rename(["altitude"])
dem_pyr = dem_pyr.clip(fc_pyr).updateMask(dem_pyr>800)

In [51]:
# Set projection and scale to avoid having multiple ones since GLO30 is an ImageCollection - apparently needed for slope computation
# See https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_DEM_GLO30
dem_corse = ee.ImageCollection("COPERNICUS/DEM/GLO30").filterBounds(fc_corse).select("DEM")
dem_corse = dem_corse.mosaic().setDefaultProjection(crs='EPSG:3035',scale=30).rename(["altitude"])
dem_corse = dem_corse.clip(fc_corse).updateMask(dem_corse>800)

In [52]:
m = geemap.Map()
m.add_layer(dem_pyr.select(["altitude"]), {"min":700, "max":3000, "palette":geecm.palettes.viridis})
m.add_layer(dem_corse.select(["altitude"]), {"min":700, "max":3000, "palette":geecm.palettes.viridis})
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## 1.2 Get S2 indices

In [53]:
for region, fc_bv, dem_bv in zip(["corse", "pyr"], [fc_corse, fc_pyr], [dem_corse, dem_pyr]):
    
    geom_bv = ee.Geometry.convexHull(fc_bv.geometry())
    
    # All indices
    s2_indices_ts = (ee.ImageCollection('COPERNICUS/S2_SR')
                        .filter(ee.Filter.calendarRange(day_beg.getRelative("day", "year"),
                                                        day_end.getRelative("day", "year"),
                                                        'day_of_year'))                    
                        .filter(ee.Filter.calendarRange(2017, 2023, 'year'))
                        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_thresh)) 
                        .filterBounds(geom_bv)
                        .scaleAndOffset()
                        .maskClouds(scaledImage=True)
                        .spectralIndices(["NDWI"])
                        .map(gee.add_s2_nari)
                        .map(gee.add_s2_ncri)
                     ).select(indices_classif+["NDWI"])


    # NARI and NCRI
    s2_ts_mm = gee.ic_monthly_median(s2_indices_ts.select(indices_classif), 
                                     month_list=[7,8,9], 
                                     first_day_of_month=15, 
                                     band_names=indices_classif)

    geemap.ee_export_image_to_asset(s2_ts_mm.clip(fc_bv).updateMask(dem_bv>700), 
                                    assetId=f"users/aguerou/lacs_sentinelles/indices/s2_indices_nari_ncri_median_ts_{region}",
                                    region=geom_bv,
                                    scale=scale,
                                    crs=projection,
                                    maxPixels=5e9,)

    # NDWI
    s2_indices_median_ndwi = s2_indices_ts.select(["NDWI"]).median()

    geemap.ee_export_image_to_asset(s2_indices_median_ndwi.clip(fc_bv).updateMask(dem_bv>700), 
                                    assetId=f"users/aguerou/lacs_sentinelles/indices/s2_indices_ndwi_median_{region}",
                                    region=geom_bv,
                                    scale=scale,
                                    crs=projection,
                                    maxPixels=5e9,)

# 2. Landcover classification

## 2.1 Parameters and datasets

Random Forest

In [3]:
training_label = "landcover"
n_trees = 50
indices_ts = ["NARI_7", "NCRI_7", 
              "NARI_8", "NCRI_8", 
              "NARI_9", "NCRI_9"]

Nomenclature

In [12]:
mapping_file = "/home/aguerou/data/land_cover/item/carto_h1b/nomenclature_h1b.txt"
mapping_name = "h1b_paper"

In [20]:
lcmap = LandCoverMap(mapping_file, name=mapping_name)
lcmap = lcmap.remove_item(col_name="Code", col_val=[8,9], in_place=False)

In [21]:
lcmap_classify = lcmap.remove_item(col_name="Code", col_val=[4, 5,6], in_place=False)

Scale and projection

In [22]:
scale = 20
projection = "EPSG:3035"

Datasets

In [23]:
ds_training = ee.FeatureCollection('users/aguerou/ice_and_life/carto_h1b/dataset_training/ds_training_g24_nari_ncri_median_ts')

In [24]:
fc_corse = ee.FeatureCollection('users/aguerou/lacs_sentinelles/ancillary/regions-20180101').filter(ee.Filter.eq('nom', 'Corse'))
fc_pyr = ee.FeatureCollection('users/aguerou/lacs_sentinelles/ancillary/pyrenees')

In [25]:
s2_indices_corse = ee.Image("users/aguerou/lacs_sentinelles/indices/s2_indices_nari_ncri_median_ts_corse")
s2_indices_pyr = ee.Image("users/aguerou/lacs_sentinelles/indices/s2_indices_nari_ncri_median_ts_pyr")

## 2.2 Random forest

Takes about 1 min per region / alps ~20min

In [30]:
for region, fc_bv, s2_indices in zip(["corse", "pyr"], [fc_corse, fc_pyr], [s2_indices_corse, s2_indices_pyr]):
    
    geom_bv = ee.Geometry.convexHull(fc_bv.geometry())  
    
    # Create RF
    rf_gee = ee.Classifier.smileRandomForest(n_trees, minLeafPopulation=2)

    # Apply random forest
    rf_mean_ts, rf_std_ts, errorMatrix_ts, df_scores = gee.rf_circular(rf_gee, 
                                                                        ds_training, 
                                                                        s2_indices, 
                                                                        indices_names=indices_ts, 
                                                                        labels_lc=list(lcmap_classify.get_type()))

    # Classif
    classif_ts = gee.classify(rf_mean_ts, 
                               lc_labels=list(lcmap_classify.get_type()), 
                               lc_values=list(lcmap_classify.get_code()), 
                               extra_class=[(lcmap_classify.get_type()[0], operator.gt, 0.25),
                                            (lcmap_classify.get_type()[0], operator.lt, 0.5),
                                            (lcmap_classify.get_type()[1], operator.gt, 0.25),
                                            (lcmap_classify.get_type()[1], operator.lt, 0.5),
                                           ], 
                               extra_class_value=lcmap.get_code()[-1])

    # Save Carto
    geemap.ee_export_image_to_asset(classif_ts.clip(fc_bv),
                                    assetId=f"users/aguerou/lacs_sentinelles/carto/carto_landcover_s2_rf_only_{region}",
                                    scale=scale,
                                    crs=projection,
                                    maxPixels=1e9,
                                    region=geom_bv)

# 3. Water classif + merge RGI & LC classif

In [14]:
scale = 20
projection = "EPSG:3035"

In [15]:
fc_corse = ee.FeatureCollection('users/aguerou/lacs_sentinelles/ancillary/regions-20180101').filter(ee.Filter.eq('nom', 'Corse'))
fc_pyr = ee.FeatureCollection('users/aguerou/lacs_sentinelles/ancillary/pyrenees')

In [16]:
rgi_alps = ee.FeatureCollection("users/aguerou/ice_and_life/carto_h1b/lia_shp/c3s_gi_rgi11_s2_2015_v2")
rgi_pyr = ee.FeatureCollection("users/aguerou/ice_and_life/carto_h1b/pyr/RGI2000-v7-G-11_france")

In [17]:
lc_corse = ee.Image("users/aguerou/lacs_sentinelles/carto/carto_landcover_s2_rf_only_corse")
lc_pyr = ee.Image("users/aguerou/lacs_sentinelles/carto/carto_landcover_s2_rf_only_pyr")

In [18]:
ndwi_corse = ee.Image("users/aguerou/lacs_sentinelles/indices/s2_indices_ndwi_median_corse")
ndwi_pyr = ee.Image("users/aguerou/lacs_sentinelles/indices/s2_indices_ndwi_median_pyr")

## 3.1 Merge classification

In [19]:
for region, lc_region, ndwi_region, rgi, fc_bv in zip(["pyr"], [lc_pyr], [ndwi_pyr], [rgi_pyr], [fc_pyr]):
      
    geom_bv = ee.Geometry.convexHull(fc_bv.geometry())  
    
    # Get mask of glacier
    glacier_region = lc_region.clip(rgi)

    # Glacier after water to avoid water on glacier surface with very low NDWi thresholds
    classif_region = lc_region.where(ndwi_region.select("NDWI")>0.15, 5).where(glacier_region.gt(-1), 4)

    # Export to asset
    geemap.ee_export_image_to_asset(classif_region.clip(fc_bv),
                                    assetId=f"users/aguerou/lacs_sentinelles/carto/carto_landcover_s2_{region}",
                                    scale=scale,
                                    crs=projection,
                                    maxPixels=1e9,
                                    region=geom_bv)

    # Export to drive
    geemap.ee_export_image_to_drive(classif_region.clip(fc_bv),
                                    folder="gee_dwnld_lacs_sentinelles",
                                    fileNamePrefix=f"carto_landcover_s2_{region}",
                                    scale=scale,
                                    crs=projection,
                                    maxPixels=1e9,
                                    region=geom_bv,
                                    formatOptions={"noData": -1})